Preprocessing - **PCA** (component=4)  
Algorithm - **KNN Classification**  
Feature - **Magnitude** **(251)**

In [3]:
# Real + Imaginary together as one feature
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


# Load dataset
measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:200]
n_comp = 4

# Train-test split
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)

trainData - (160, 15, 251, 2)
testData - (40, 15, 251, 2)


In [ ]:
# -------------------------------------------------- Preprocessing ------------------------------------------
def apply_pca(data, n_components):
    # data: (samples, 251, 2)
    reshaped_data = data.reshape(data.shape[0], -1)  # Now shape is (samples, 251*2) -> (12800, 502)
    
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)  # (samples, 251*2) -> (12800, 502)
    print(f"data_scaled: {data_scaled.shape}")
    
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data_scaled)  # (samples, n_components)
    
    return data_pca, scaler, pca

# -------- Train --------
# Feature Extraction
train_alice_cirs = trainCIR[:, alice_channel, :, :]
train_eve_cirs = trainCIR[:, eve_channel, :, :]

train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_cirs = np.vstack((train_alice_magnitude, train_eve_magnitude))
train_cirs_pca, scaler, pca = apply_pca(train_cirs, n_components=n_comp)

# Labels
train_alice_labels = np.zeros(train_alice_cirs.shape[0])
train_eve_labels = np.ones(train_eve_cirs.shape[0])
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# -------- Test --------
# Feature Extraction
test_alice_cirs = testCIR[:, alice_channel, :, :]
test_eve_cirs = testCIR[:, eve_channel, :, :]

test_alice_magnitude = np.abs(test_alice_cirs[..., 0] + 1j * test_alice_cirs[..., 1])
test_eve_magnitude = np.abs(test_eve_cirs[..., 0] + 1j * test_eve_cirs[..., 1])
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))

# Labels
test_alice_label = np.zeros(test_alice_cirs.shape[0])
test_eve_labels = np.ones(test_eve_cirs.shape[0])
test_labels = np.hstack((test_alice_label, test_eve_labels))

# Reshape
reshaped_test_cirs = test_cirs.reshape(test_cirs.shape[0], -1)
test_cirs_scaled = scaler.transform(reshaped_test_cirs)

# PCA
test_cirs_pca = pca.transform(test_cirs_scaled)


train_cirs: (320, 251)
data_scaled: (320, 251)


In [7]:

# ----------------- Classification -----------------
# Train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_cirs_pca, train_labels)

# Predict on test data
predictions = knn.predict(test_cirs_pca)


In [8]:
# ----------------- Evaluation -----------------
# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Alarm Rate (FAR)
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0

# Gamma calculation
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0

# Authentication Rate (AR)
denominator = (tp + fn) + gamma * (tn + fp)
AR = (tp + gamma * tn) / denominator if denominator > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 57.19%
tp: 462
tn: 453
fp: 347
fn: 338
MDR: 0.43375
FAR: 0.4225
AR: 0.571875


In [6]:
import numpy as np

def pca_reduce_samples(data, n_components):
    """
    Custom PCA function to reduce the number of samples in the data.
    
    Parameters:
    - data: Input data of shape (samples, features)
    - n_components: The desired number of reduced samples (principal components)
    
    Returns:
    - reduced_data: Data with reduced number of samples but same number of features
    """
    # Step 1: Standardize the data (mean center)
    # Mean centering is crucial for PCA
    mean_centered_data = data - np.mean(data, axis=0)  # Shape: (samples, features)
    
    # Step 2: Compute the covariance matrix of the data
    # Since we want to reduce the samples, calculate covariance matrix considering rows as samples
    covariance_matrix = np.dot(mean_centered_data, mean_centered_data.T) / (data.shape[1] - 1)  # Shape: (samples, samples)
    
    # Step 3: Perform eigen decomposition of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)  # Shape: (samples, samples)
    
    # Step 4: Sort eigenvectors by descending eigenvalues
    # argsort gives the indices that would sort the eigenvalues in ascending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    
    # Step 5: Select the top n_components eigenvectors (principal components)
    selected_eigenvectors = sorted_eigenvectors[:, :n_components]  # Shape: (samples, n_components)
    
    # Step 6: Project the original data onto the selected components
    # Projection here reduces the number of samples
    reduced_data = np.dot(selected_eigenvectors.T, mean_centered_data)  # Shape: (n_components, features)
    
    return reduced_data  # Shape: (n_components, features)

# Example usage:
data = np.random.rand(100, 20)  # 100 samples, 20 features
n_components = 10  # Reduce to 10 samples

reduced_data = pca_reduce_samples(data, n_components)
print("Original shape:", data.shape)  # Output: (100, 20)
print("Reduced shape:", reduced_data.shape)  # Output: (10, 20)


Original shape: (100, 20)
Reduced shape: (10, 20)
